In [1]:
from numpy import pi, linspace, array, where, arcsin, sqrt, diff
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame, ecliptic_frame
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Slider, Select, DatePicker, MultiSelect, MultiChoice

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
ts = load.timescale()
colors = ["red", "green", "blue", "yellow", "purple", "orange", "black", "magenta", "pink"]
eph_s_name = 'de421.bsp'
# eph_s_name = 'de422.bsp'
t_start = ts.tt(2000, 6, 21, 0, 0, 0)
t_end = ts.tt(2002, 6, 21, 0, 0, 0)
n_points = 1000
eph_s = load(eph_s_name)

In [4]:
def f_cross_zero():
    def zero(t):
        _, _, z = (moon.at(t) - earth.at(t)).frame_xyz(ecliptic_J2000_frame).au
        return z > 0
    zero.step_days = 10.0
    return zero

In [5]:
times = ts.tt_jd(linspace(t_start.tt, t_end.tt, n_points))
moon = eph_s["moon"]
earth = eph_s["earth"]
x, y, z = (moon.at(times) - earth.at(times)).frame_xyz(ecliptic_J2000_frame).au
angle_z = arcsin(z / sqrt(x*x + y*y + z*z)) * 180/pi
ecliptic_cross = find_discrete(t_start, t_end, f_cross_zero())
single_nodes = (ecliptic_cross[0][ecliptic_cross[1] == 1])
diff(single_nodes)

array([np.float64(27.450215376447886), np.float64(27.367647329345345),
       np.float64(27.178101784083992), np.float64(27.0248243464157),
       np.float64(27.06826721318066), np.float64(27.27924930304289),
       np.float64(27.457857329864055), np.float64(27.455239467788488),
       np.float64(27.274086337536573), np.float64(27.068128073122352),
       np.float64(27.02894919505343), np.float64(27.1719763353467),
       np.float64(27.347276139073074), np.float64(27.425588049925864),
       np.float64(27.361121197231114), np.float64(27.18881528545171),
       np.float64(27.03953703586012), np.float64(27.05947137903422),
       np.float64(27.230642876122147), np.float64(27.389249107800424),
       np.float64(27.39985710894689), np.float64(27.250323409214616),
       np.float64(27.074507421813905), np.float64(27.04188592545688),
       np.float64(27.156932966783643), np.float64(27.29128197301179)],
      dtype=object)

In [6]:
fig = figure(title="Moon", x_axis_label="x", y_axis_label="y", width=1100)

fig.line(times.tt, z, legend_label="z", color="blue")
fig.scatter(times.tt, z, legend_label="z", color="red")
fig.scatter(times.tt, angle_z/20000, legend_label='theta(deg)/20000', color="green")

show(fig)